In [ ]:
import pyterrier as pt
import matplotlib.pyplot as plt
from os import listdir
from os.path import join, isdir
import sys
if not pt.started():
    pt.init()

In [ ]:
files = "archivo_salida.trec"
files

In [ ]:


indexer = pt.TRECCollectionIndexer("/home/franco/Documentos/UNLu/Recuperacion de Informacion/TP1/RI/TP2/P9/index", verbose=True, overwrite=True, meta={"docno":20, "filename":512})
indexref = indexer.index(files)



In [ ]:
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

In [ ]:
tf_idf = pt.BatchRetrieve(index, num_results=10, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, num_results=10, wmodel="BM25")

In [ ]:
queries_sin_frec = pt.io.read_topics("queries_CISI.trec")
queries_con_frec = pt.io.read_topics("queriesfrec_CISI.trec")

qrels = pt.io.read_qrels("qrels_CISI")

In [ ]:
queries_con_frec

In [ ]:
print(qrels)

In [ ]:
## Queries con Frec

eval_metrics = [
    "num_q", "num_ret", "num_rel", "num_rel_ret", "map", 
    "Rprec", "bpref", "recip_rank", "iprec_at_recall_0.00", 
    "iprec_at_recall_0.10", "iprec_at_recall_0.20", "iprec_at_recall_0.30", 
    "iprec_at_recall_0.40", "iprec_at_recall_0.50", "iprec_at_recall_0.60", 
    "iprec_at_recall_0.70", "iprec_at_recall_0.80", "iprec_at_recall_0.90", 
    "iprec_at_recall_1.00", "P_5", "P_10", "P_15", "P_20", "P_30", 
    "P_100", "P_200", "P_500", "P_1000"
]

# Ejecutar el experimento solo para el sistema TF-IDF
resultados_tfidf = pt.Experiment(
    [tf_idf],
    queries_con_frec,
    qrels,
    eval_metrics=eval_metrics,
    names=["TF_IDF"]
)
resultados_tfidf

In [ ]:
# Mostrar Tabla de P@X
tabla = resultados_tfidf.melt(value_vars=["P_5", "P_10", "P_15", 
                                    "P_20", "P_30", "P_100", 
                                    "P_200", "P_500", "P_1000"], var_name="P@X", value_name="Value")
tabla

In [ ]:
df_11puntos = resultados_tfidf.melt(value_vars=["iprec_at_recall_0.10", "iprec_at_recall_0.20", "iprec_at_recall_0.30", 
                                    "iprec_at_recall_0.40", "iprec_at_recall_0.50", "iprec_at_recall_0.60", 
                                    "iprec_at_recall_0.70", "iprec_at_recall_0.80", "iprec_at_recall_0.90", 
                                    "iprec_at_recall_1.00"], var_name="Recall", value_name="Precision")

df_11puntos['Recall'] = df_11puntos['Recall'].apply(lambda x: float(x.split('_')[-1]))

df_11puntos

In [ ]:
# Crear el gráfico de líneas
plt.figure(figsize=(10, 6))  # Establecer el tamaño del gráfico
plt.plot(df_11puntos['Recall'], df_11puntos['Precision'], marker='o', linestyle='-')  # Graficar Recall vs Precision
plt.title('Recall vs Precision - Con Frecuencias')  # Establecer el título del gráfico
plt.xlabel('Recall')  # Etiqueta del eje x
plt.ylabel('Precision')  # Etiqueta del eje y
plt.grid(True)  # Activar la cuadrícula en el gráfico
plt.show()  # Mostrar el gráfico

In [ ]:
## Queries sin Frec

# Ejecutar el experimento solo para el sistema TF-IDF
resultados_tfidf_sin_frecs = pt.Experiment(
    [tf_idf],
    queries_sin_frec,
    qrels,
    eval_metrics=eval_metrics,
    names=["TF_IDF"]
)
resultados_tfidf_sin_frecs

In [ ]:
# Mostrar Tabla de P@X
tabla = resultados_tfidf_sin_frecs.melt(value_vars=["P_5", "P_10", "P_15", 
                                    "P_20", "P_30", "P_100", 
                                    "P_200", "P_500", "P_1000"], var_name="P@X", value_name="Value")
tabla

In [ ]:
# Mostrar R–P en los 11 puntos standard
df_11puntos_sin_frec = resultados_tfidf_sin_frecs.melt(value_vars=["iprec_at_recall_0.10", "iprec_at_recall_0.20", "iprec_at_recall_0.30", 
                                    "iprec_at_recall_0.40", "iprec_at_recall_0.50", "iprec_at_recall_0.60", 
                                    "iprec_at_recall_0.70", "iprec_at_recall_0.80", "iprec_at_recall_0.90", 
                                    "iprec_at_recall_1.00"], var_name="Recall", value_name="Precision")
# Modificar los valores de recall
df_11puntos_sin_frec['Recall'] = df_11puntos_sin_frec['Recall'].apply(lambda x: float(x.split('_')[-1]))

df_11puntos_sin_frec

In [ ]:
# Crear el gráfico de líneas
plt.figure(figsize=(10, 6))  # Establecer el tamaño del gráfico
plt.plot(df_11puntos_sin_frec['Recall'], df_11puntos_sin_frec['Precision'], marker='o', linestyle='-', label='Sin Frecs')  # Graficar la primera línea
plt.plot(df_11puntos['Recall'], df_11puntos['Precision'], marker='x', linestyle='-', label='Con Frecs')  # Graficar la segunda línea
plt.title('Recall vs Precision')  # Establecer el título del gráfico
plt.xlabel('Recall')  # Etiqueta del eje x
plt.ylabel('Precision')  # Etiqueta del eje y
plt.grid(True)  # Activar la cuadrícula en el gráfico
plt.legend()  # Mostrar leyenda
plt.show()  # Mostrar el gráfico

In [ ]:
def graficar_recall(df):
    plt.plot(df.index, df['Recall'], marker='o', linestyle='-', label='Recall')

def graficar_precision(df):
    plt.plot(df.index, df['Precision'], marker='o', linestyle='-', label='Precision')

# Llamamos a las funciones para trazar las líneas
graficar_recall(df_11puntos_sin_frec)
graficar_precision(df_11puntos_sin_frec)

# Añadir etiquetas, título, leyenda y mostrar el gráfico
plt.xlabel('Número de documentos')
plt.ylabel('Valor')
plt.title('Recall y Precisión para documentos')
plt.legend()
plt.grid(True)
plt.show()